In [1]:
from pycocotools.coco import COCO
from sklearn import tree, metrics
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
import json
from LoadLocalCOCO import LoadLocalCOCO as llc
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def flatten_list(qlist):
    return [y for x in qlist for y in x]

In [3]:
train_instance="annotations/instances_train2017.json"
val_instance="annotations/instances_val2017.json"


In [4]:
coco_path = "D:/FSR/COCO/train2017"

In [5]:
classes = ['dog', 'cat']

In [6]:
coco_train = llc(train_instance, coco_path)
coco_val = llc(val_instance, coco_path)

In [7]:
# train_n = [coco_train.get_number_of_images_per_category(x) for x in classes]
# val_n = [coco_val.get_number_of_images_per_category(x) for x in classes]
# print(f'Training:> \t Dog: {train_n[0]} \t Cat: {train_n[1]}')
# print(f'Validation:> \t Dog: {val_n[0]} \t Cat: {val_n[1]}')

In [8]:
data_train = []
for category in classes:
    data = coco_train.get_cropped_images_flatten_and_resized(category, resize=(64, 64))
    data_train.append(data)

5508it [05:51, 15.69it/s]
2it [00:00, 16.54it/s]Broken images: [7125, 16775, 24247, 24247, 55296, 47386, 69468, 35613, 77325, 12696, 22427, 26321, 54957, 790, 1360, 55478, 42341, 57631, 2400, 10276, 94049, 58741, 5782, 44613, 58741, 24159, 44613, 29176] images
4768it [04:57, 16.02it/s]Broken images: [5617, 711, 39171, 9413, 46749, 18211, 26375, 26375, 26375, 66041] images



In [9]:
np.shape(data_train[0])

(5480, 12288)

In [10]:
labels = len(data_train[0])*[1]+len(data_train[1])*[0]

In [11]:
np.shape(labels)

(10238,)

In [12]:
np.shape(data_train)

(2,)

In [13]:
data_train = flatten_list(data_train)

In [14]:
np.shape(data_train)

(10238, 12288)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_train, labels)

In [18]:
pca = PCA(n_components = 25)
pca.fit(X_train)
reduced_X_train_pca, reduced_X_test_pca = pca.transform(X_train), pca.transform(X_test)

In [28]:
np.shape(reduced_X_train)

(7678, 24)

In [29]:
np.shape(X_train)

(7678, 67500)

In [19]:
ica = FastICA(n_components=7, random_state=0, whiten=True)

In [22]:
help(FastICA)

Help on class FastICA in module sklearn.decomposition._fastica:

class FastICA(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  FastICA(n_components=None, algorithm='parallel', whiten=True, fun='logcosh', fun_args=None, max_iter=200, tol=0.0001, w_init=None, random_state=None)
 |  
 |  FastICA: a fast algorithm for Independent Component Analysis.
 |  
 |  Read more in the :ref:`User Guide <ICA>`.
 |  
 |  Parameters
 |  ----------
 |  n_components : int, optional
 |      Number of components to use. If none is passed, all are used.
 |  
 |  algorithm : {'parallel', 'deflation'}
 |      Apply parallel or deflational algorithm for FastICA.
 |  
 |  whiten : boolean, optional
 |      If whiten is false, the data is already considered to be
 |      whitened, and no whitening is performed.
 |  
 |  fun : string or function, optional. Default: 'logcosh'
 |      The functional form of the G function used in the
 |      approximation to neg-entropy. Could be either 'logcosh', 'ex

In [20]:
ica.fit(X_train)

FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
        n_components=7, random_state=0, tol=0.0001, w_init=None, whiten=True)

In [21]:
reduced_X_train_ica = ica.transform(X_train)

In [22]:
reduced_X_test_ica = ica.transform(X_test)

In [16]:
svd = TruncatedSVD(n_components = 25)
svd.fit(X_train)

TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
             random_state=None, tol=0.0)

In [17]:
reduced_X_train_svd, reduced_X_test_svd = svd.transform(X_train), svd.transform(X_test)